In [1]:
%matplotlib inline 
import matplotlib.pyplot as plt
import pandas as pd 
from datetime import datetime
from tqdm import tqdm
import pandas as pd 
import numpy as np 
import cv2 
import os 
import pickle 
import random
import shutil 
import uuid
# Image augmentation 
import albumentations as A

In [2]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
data_path = 'Data'

# Original Path 
original_train_path = os.path.join(data_path, 'train') 
original_test_path = os.path.join(data_path, 'test') 

# Augmented Path
augmented_train_path = os.path.join(data_path , 'augmented_train')
augmented_test_path = os.path.join(data_path , 'augmented_test')
augmented_validation_path = os.path.join(data_path, 'augmented_validation')

In [4]:
CLASSES = os.listdir(original_train_path)
IMG_SIZE = 256
AUGMENTATION_MULTIPLEX = 4
'Classes : '+', '.join(CLASSES)

'Classes : blue, no helmet, not helmet, red, white, yellow'

In [5]:
def create_new_dir(new_data):
  if not os.path.exists(new_data):  
    os.mkdir(new_data)
  else: 
    shutil.rmtree(new_data)
    os.mkdir(new_data)

def random_image_name():
    img_name = f'{uuid.uuid4().hex}_helmet.jpg'
    return img_name

In [6]:
create_new_dir(augmented_train_path)
create_new_dir(augmented_test_path)
create_new_dir(augmented_validation_path)


def image_augmentation(save_dir, image_gen):
  print(save_dir)
  for image_dir in tqdm(image_gen):
    
    # Copy and paste original image 
    save_file_dir = os.path.join(save_train ,random_image_name())
    shutil.copy(image_dir, save_file_dir)

    # Compress and Augment images 
    img_array = cv2.imread( image_dir,cv2.COLOR_BGR2RGB)
    img_array = cv2.normalize(img_array , None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    img_array = cv2.resize( img_array, ( 256, 256)) 
    
    for _ in range(AUGMENTATION_MULTIPLEX):

      # Augmentation properties

      transform = A.Compose([
          A.RandomCrop(width=256, height=256),
          A.HorizontalFlip(p=0.8),
          A.RandomBrightnessContrast(p=0.5),
          A.RandomRotate90()
      ])
      

      # Augment image
      transformed = transform(image=img_array )
      img_array_resize = cv2.resize( transformed["image"], ( IMG_SIZE, IMG_SIZE))
      save_image = cv2.convertScaleAbs(img_array_resize, alpha=(256.0))

      # Save augmented image 
      save_file = os.path.join( save_dir, random_image_name())
      cv2.imwrite(save_file , save_image)

In [7]:
for classes in CLASSES:

  # Original Train dir and Train images 
  class_train_dir = os.path.join(original_train_path, classes)
  class_train =[ os.path.join(class_train_dir,i) for i in os.listdir(class_train_dir)]

  # Original Test dir and Test imagesa 
  class_test_dir = os.path.join(original_test_path, classes)
  class_test =  [ os.path.join(class_test_dir,i) for i in os.listdir(class_test_dir)]

  # Concat and shuffle 
  images = np.array(class_train+class_test)
  np.random.shuffle(images)

  # Split the images into 3 segements ( train 80%, test 15%, validation 5% )
  
  train_count = int(len(images)*0.80)
  test_count =  int(len(images)*0.15)
  validation_count =  int(len(images)*0.05)

  train = images[:train_count]
  test = images[train_count:train_count+test_count]
  validation = images[train_count+test_count:train_count+test_count+validation_count]

  # Shufle splitted segments 
  np.random.shuffle(train)
  np.random.shuffle(test)
  np.random.shuffle(validation)

  # Images Dir 
  save_train = os.path.join(augmented_train_path, classes)
  save_test = os.path.join(augmented_test_path , classes)
  save_validation = os.path.join(augmented_validation_path, classes)

  if not os.path.exists(save_train): os.mkdir(save_train)
  if not os.path.exists(save_test): os.mkdir(save_test)
  if not os.path.exists(save_validation): os.mkdir(save_validation)

  # Image augmentation 
  image_augmentation( save_train ,train)
  image_augmentation( save_test ,test)
  image_augmentation( save_validation,  validation)

  print(f'--- [ {classes} ] ----')
  print(f'Original train: ',len(class_train))
  print(f'Original test: ',len(class_test))
  print(f'Splitted train: ', len(train) )
  print(f'Splitted test: ', len(test) )
  print(f'Splitted validation: ', len(validation))
  print('-------------------------\n')

Data\augmented_train\blue


100%|██████████| 1200/1200 [00:34<00:00, 34.89it/s]


Data\augmented_test\blue


100%|██████████| 225/225 [00:07<00:00, 31.08it/s]


Data\augmented_validation\blue


...

Data\augmented_test\yellow


100%|██████████| 225/225 [00:06<00:00, 32.57it/s]


Data\augmented_validation\yellow


100%|██████████| 75/75 [00:02<00:00, 33.36it/s]

--- [ yellow ] ----
Original train:  1200
Original test:  300
Splitted train:  1200
Splitted test:  225
Splitted validation:  75
-------------------------

